In [24]:
@interact
def oscTest(h = slider( 0, 1, 0.1, 0.5), 
            s = slider(-2, 2, 0.1, 1),
            d = slider(-12, 12, 0.1, 0)):
    a1 = (s+d)/h           # correct (maybe up to a constant)
    b1 = 1/(4*h) - a1*h/2  # normalizes singed area of 1st section to 1/4
    a2 = (s-d)/h
    b2 = -1/4*(2*a2*h^2 - 2*a2 - 1)/(h - 1)
    y  = piecewise([ ((0,h),a1*x+b1), ((h,1),a2*x+b2) ])
    p = plot(y(x), 0, 1)
    #p.show()
    
# idea: have 3 user params (h (transition point), slope1, slope2) and 2 
# constraints: signed area of 1st half-wave +1/4, 2nd: -1/4
# slope1/2 directly determine a1/a2 

SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIG9zY1Rlc3QgYXQgMHg2ZmYyNzg1N2Q3MD4gd2l0aCAzIHdpZGdldHMKICBoOiBUcmFuc2Zvcm1GbG9hdFNsaWRlcih2YWx1ZT0wLjUsIGTigKY=


a1, a2 are given from user params: a1 = (s+d)/h, a2 = (s-d)/h, where s is the desired average slope and d is the slope-difference between 1st and 2nd section. h is given as a user parameter anyway. What remains to calculate the two offsets b1,b2. To do this, we apply the following constraints:  
(1)  no DC: integral over the waveform from 0 to 1 must be 0
(2)  fixed energy: integral over the square of the waveform must have given value E

In [9]:
var("x a1 b1 a2 b2 h E")
y1(x) = a1*x + b1  # 1st segment
y2(x) = a2*x + b2  # 2nd segment
A = integral( y1(x),    x, 0, h) + integral( y2(x),    x, h, 1) # signed area
E = integral(abs(y1(x)), x, 0, h) + integral(abs(y2(x)), x, h, 1) # energy
# gives:
# A = 1/2*a1*h^2 - 1/2*a2*h^2 + b1*h - b2*h + 1/2*a2 + b2
# E = 1/3*a1^2*h^3 - 1/3*a2^2*h^3 + a1*b1*h^2 - a2*b2*h^2 
#     + b1^2*h - b2^2*h + 1/3*a2^2 + a2*b2 + b2^2
# A should be zero (no DC) and E should be 1/2 (energy of saw-wave with
# amplitude 1), so we get two equations and solve them for b1,b2:
eq1 = 0   == 1/2*a1*h^2 - 1/2*a2*h^2 + b1*h - b2*h + 1/2*a2 + b2
eq2 = 1/12 == 1/3*a1^2*h^3 - 1/3*a2^2*h^3 + a1*b1*h^2 - a2*b2*h^2 + b1^2*h - b2^2*h + 1/3*a2^2 + a2*b2 + b2^2
solve([eq1,eq2],[b1,b2])
# ok - this works - but the solutions are complicated and contain square roots

[[b1 == -1/6*(3*a1*h^2 + sqrt(3*(a1^2 - a2^2)*h^5 - 18*a2^2*h^3 - 3*(a1^2 - 4*a2^2)*h^4 + 3*(4*a2^2 - 1)*h^2 - 3*(a2^2 - 1)*h))/h, b2 == -1/6*(3*a2*h^2 - 3*a2 + sqrt(3*(a1^2 - a2^2)*h^5 - 18*a2^2*h^3 - 3*(a1^2 - 4*a2^2)*h^4 + 3*(4*a2^2 - 1)*h^2 - 3*(a2^2 - 1)*h))/(h - 1)], [b1 == -1/6*(3*a1*h^2 - sqrt(3*(a1^2 - a2^2)*h^5 - 18*a2^2*h^3 - 3*(a1^2 - 4*a2^2)*h^4 + 3*(4*a2^2 - 1)*h^2 - 3*(a2^2 - 1)*h))/h, b2 == -1/6*(3*a2*h^2 - 3*a2 - sqrt(3*(a1^2 - a2^2)*h^5 - 18*a2^2*h^3 - 3*(a1^2 - 4*a2^2)*h^4 + 3*(4*a2^2 - 1)*h^2 - 3*(a2^2 - 1)*h))/(h - 1)]]

In [15]:
(-1/6*(3*a1*h^2 + sqrt(3*(a1^2 - a2^2)*h^5 - 18*a2^2*h^3 - 3*(a1^2 - 4*a2^2)*h^4 + 6*(2*a2^2 - 3)*h^2 - 3*(a2^2 - 6)*h))/h).full_simplify()

-1/6*(3*a1*h^2 + sqrt(3*(a1^2 - a2^2)*h^5 - 18*a2^2*h^3 - 3*(a1^2 - 4*a2^2)*h^4 + 6*(2*a2^2 - 3)*h^2 - 3*(a2^2 - 6)*h))/h

In [ ]:
E = integral(abs(y1(x)), x, 0, h) + integral(abs(y2(x)), x, h, 1)
eq1 = 0   == 1/2*a1*h^2 - 1/2*a2*h^2 + b1*h - b2*h + 1/2*a2 + b2
eq2 = 1/2 == 1/2*(a1*h + b1)*abs(a1*h + b1)/a1 - 1/2*(a2*h + b2)*abs(a2*h + b2)/a2 + 1/2*(a2 + b2)*abs(a2 + b2)/a2 - 1/2*b1*abs(b1)/a1
solve([eq1,eq2],[b1,b2])

In [8]:
#2 * integral(2*x, x, 0, 1/2) # energy of unit amplitude saw (wrong)
2 * integral(x^2, x, 0, 1/2) # energy of unit amplitude saw

1/12